In [1]:
#FOLDER TO PULL DATA FROM/SAVE TO:
FOLDER = 'Data4'

In [2]:
from os.path import join
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import trange
import torch.nn.functional as F
#[8]

trained_model = GPT2LMHeadModel.from_pretrained('gpt2')
trained_model.load_state_dict(torch.load(join(FOLDER, 'trained_model.pt')))
#print(loaded_model)
#model.eval()

#trained_model = trained_model.cuda()

<All keys matched successfully>

In [3]:
import pandas as pd

test_set = pd.read_csv(join(FOLDER,'test_set.csv'))
print(test_set)

     index                                         para_array  \
0      405  ["'All", 'the', "same,'", 'said', 'Frodo,', "'...   
1     1511  ["'Dark", "times,'", 'said', 'Strider.', "'But...   
2     2783  ['He', 'named', 'the', 'nameless', 'hills', 'a...   
3     1936  ['`I', 'can', 'take', 'you', 'to', 'them,', "s...   
4     8506  ['With', 'a', 'long', 'knife', 'in', 'his', 'l...   
..     ...                                                ...   
863   1902  ['`What', 'would', 'they', 'have', 'done', 'to...   
864   6049  ['`Which', 'way', 'do', 'we', 'go', 'from', "h...   
865   2175  ['`Then', 'it', 'belongs', 'to', 'you,', 'and'...   
866   1421  ['The', 'landlord', 'jumped', 'with', 'surpris...   
867    195  ['Frodo', 'took', 'the', 'envelope', 'from', '...   

                                              para_raw              book  \
0    'All the same,' said Frodo, 'even if Bilbo cou...  Corpus\lotr1.txt   
1    'Dark times,' said Strider. 'But for the prese...  Corpus\lotr

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [5]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    
    #prompt_len = len(prompt.split())
    #print(prompt_len)
    
    
    device = torch.device("cuda")
    model.eval()
    generated_num = 0
    generated_list = []
    

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt), device=device).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(torch.Tensor.cpu(generated.squeeze()).numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    #print(output_text)
                    #print(output_list)
                    #print(generated_list)
                    #print(tokenizer.decode(output_list[0]))
                    break
            
            if not entry_finished:
                output_list = list(torch.Tensor.cpu(generated.squeeze()).numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
                #print(output_text)
                #print(output_list)
                #print(generated_list)
                #print(tokenizer.decode(output_list[0]))
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data, entry_len = 30):
    
    generated_para = []
    for i in range(len(test_data)):
    #for i in range(5):
        x = generate(trained_model.to('cuda'), tokenizer, test_data['test_input'][i], entry_count=1, entry_length = entry_len)
        generated_para.append(x)
    return generated_para



In [21]:
#generate a sample sentence
text = "Justin and tim"
x = generate(trained_model.to('cuda'), tokenizer, text, entry_count=1)
print(x[0])

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]

Justin and tim truss were in charge of his business. If we get a worm, or they do, we have got something better to do than kill the Worm<|endoftext|>


In [19]:
#Run on testing data to generate test results 
generated_para = text_generation(test_set, entry_len = 30)
#print(generated_para)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [8]:
#save testing data results for comparison
gen_results = []
for elem in generated_para:
    gen_results.append(elem[0])
test_set['gen_results'] = gen_results
print(test_set['gen_results'])
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
import os
path = os.path.abspath(os.getcwd())
#print(path)
test_set.to_csv(join(path, FOLDER, 'test_set.csv'),index=False)
print("saved to " + join(path, FOLDER, 'test_set.csv'))

0      'All the same,' said Frodo, 'even if Bilbo cou...
1      'Dark times,' said Strider. 'But for the prese...
2      He named the nameless hills and the road paved...
3      `I can take you to them, sir,' said Sam. `It's...
4      With a long knife in his left hand and a club ...
                             ...                        
863    `What would they have done to me?' asked Frodo...
864    `Which way do we go from here?' asked Frodo. `...
865    `Then it belongs to you, and not to me at all!...
866    The landlord jumped with surprise. 'You!' he c...
867    Frodo took the envelope from the mantelpiece, ...
Name: gen_results, Length: 868, dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~
saved to C:\Users\tofug\Anaconda_Projects\Coen 140\final_project\Data4\test_set.csv


In [12]:
#compare a specific paragraph
i = 1
print(test_set['para_raw'][i])
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print(test_set['gen_results'][i])

'Dark times,' said Strider. 'But for the present you may be left in peace, when you have got rid of us. We will leave at once. Never mind about breakfast: a drink and a bite standing will have to do. We shall be packed in a few minutes.'
~~~~~~~~~~~~~~~~~~~~~~~~~
'Dark times,' said Strider. 'But for the present you may be left in peace, when you have got rid of us. We will leave at once. Never mind about breakfast: a drink and a bite standing beside us, and the Merry-Go-Round. Now, if you say so, and think it right, we will. You can have our<|endoftext|>
